In [ ]:
pip install transformers -q

     |████████████████████████████████| 5.5 MB 5.3 MB/s 
     |████████████████████████████████| 163 kB 41.3 MB/s 
     |████████████████████████████████| 7.6 MB 41.7 MB/s 


In [ ]:
from transformers import GPT2Tokenizer, GPT2Model
import torch
import pandas as pd
from google.colab import data_table
from tqdm.notebook import tqdm
data_table.enable_dataframe_formatter()

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2')

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

In [ ]:
input_sentence = "Specifically, we train GPT-3, an autoregressive language model, with 175 billion parameters,"
input_embedding = tokenizer(input_sentence, return_tensors="pt").input_ids

embedding_matrix = model.wte.weight
print(f"emb shape {embedding_matrix.shape}")

# print(type(input_embedding), input_embedding)
model_output = model(input_embedding, output_hidden_states=True)
hidden_states = model_output.hidden_states

emb shape torch.Size([50257, 768])


In [ ]:
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism', 'soc.religion.christian',
              'comp.graphics', 'sci.med']
              
twenty_train = fetch_20newsgroups(subset='train',
    categories=categories, shuffle=True, random_state=42)


print(twenty_train.data[0]) # pour afficher le premier texte
len(twenty_train.target) # pour afficher le premier label

From: sd345@city.ac.uk (Michael Collier)
Subject: Converting images to HP LaserJet III?
Nntp-Posting-Host: hampton
Organization: The City University
Lines: 14

Does anyone know of a good way (standard PC application/PD utility) to
convert tif/img/tga files into LaserJet III format.  We would also like to
do the same, converting to HPGL (HP plotter) files.

Please email any response.

Is this the correct group?

Thanks in advance.  Michael.
-- 
Michael Collier (Programmer)                 The Computer Unit,
Email: M.P.Collier@uk.ac.city                The City University,
Tel: 071 477-8000 x3769                      London,
Fax: 071 477-8565                            EC1V 0HB.



2257

In [ ]:
data_cn = []

batches = []
for i in range(0, len(twenty_train.data) - 1, 32):
  maxlength = 0
  batch = []
  for j in range(32):
    batch.append(tokenizer(twenty_train.data[i*32 + j], return_tensors="pt").input_ids)
    maxlength = max(maxlength, len(batch[-1]))

  batch_tensor = torch.zeros((32, maxlength))
  for j in range(32):
    sentence_ids = batch[j][0]
    print(embeded.shape)
    print(batch_tensor[j, :len(embeded)].shape)
    batch_tensor[j, :len(embeded)] = embeded
  
  batches.append(batch_tensor)


torch.Size([276])
torch.Size([1])


RuntimeError: ignored

In [ ]:
print(tokenizer(twenty_train.data[0], return_tensors="pt").input_ids.shape)

torch.Size([1, 276])


In [ ]:
for batch in tqdm(batches):
    embeded_batch = tokenizer(batch, return_tensors="pt").input_ids
    model_output = model(embeded_batch, output_hidden_states=True)
    hidden_states = model_output.hidden_states
    for i, partial_output in enumerate(hidden_states) :
        try :
          data_cn[i].append(partial_output)
        except :
          data_cn.append([partial_output])

In [ ]:
print(tokenizer.pad_token_type_id)

0


# Questions

## What is your strategy to use the internal states of gpt-2 as features for classification?
Probing : on entraine un classifier à classiifer les journeaux en fonction des inputs de GPT-2
## Propose 2 other strategies that won't work.
- 
##Try to predict the score of each layer at classifying the fetch_20newsgroups dataset.
- 0.5 >> 0.7 >> 0.5
##Implement and check your prediction.


In [3]:
from sklearn.linear_model import LogisticRegression
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [4]:
# Entrée : 1 hidden_state : nb token max * embeding_size 
# Sortie : 4 classes

# X entrée, y sortie
def test_classifier(x_train, y_train, x_val, y_val):
  model = LogisticRegression(verbose=True).fit(x_train, y_train)
  prediction = model.predict(x_val)
  score = (prediction == y_val).sum() / len(y_val)
  return score

def test_layers(layers_Xs, target_y):
  results = []
  y_train, y_val = train_test_split(target_y, shuffle=False)
  for X in layers_Xs[1:]:
    x_train, x_val = train_test_split(X, shuffle=False)
    results.append(test_classifier(x_train, y_train, x_val, y_val))
  plt.plot(results)
  return results